In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
class FRIED():

    def __init__(self, mstar=0.1, pah=0.1, dust=True):
        
        self.mstar = mstar
        self.pah_ratio = pah
        self.dust_grow = dust

        if self.dust_grow == True:
            suffix = 'growth'
        else:
            suffix = 'ism'

        # check if self.mstar is in the range of the expected value
        # if not choose the nearest value! 
        # and print a warning!

        frac_m, whole_m = math.modf(self.mstar)
        frac_m = int(frac_m*10)
        whole_m = int(whole_m)
        self.mstar = f'{whole_m}p{frac_m}'

        frac_p, whole_p = math.modf(self.pah_ratio)
        frac_p = int(frac_p*10)
        whole_p = int(whole_p)
        self.pah_ratio  = f'{whole_p}p{frac_p}'
        
        self.name = f'FRIEDV2_{self.mstar}Msol_fPAH{self.pah_ratio}_{suffix}.dat'
        
    def read_data(self):

        df = pd.read_csv('./data/'+self.name, names = ['Host star mass [Msol]', 'Disc outer radius [au]', 
                                'Surface density at 1au [gcm-2]', 'Surface density at disc outer edge [gcm-2]',
                                'FUV field strength at outer edge of grid [G0]', 'Mass loss rate [log10(Msol/yr)]'])
        return df
        

In [3]:
f = FRIED(mstar = 1.5, pah= 0.5, dust = False)

In [4]:
data = f.read_data()

In [5]:
f.name

'FRIEDV2_1p5Msol_fPAH0p5_ism.dat'

In [6]:
data

,Host star mass [Msol],Disc outer radius [au],Surface density at 1au [gcm-2],Surface density at disc outer edge [gcm-2],FUV field strength at outer edge of grid [G0],Mass loss rate [log10(Msol/yr)]
0,1.5,500.0,1.0,0.002000,1.0,-10.725531
1,1.5,250.0,1.0,0.004000,1.0,-13.446023
2,1.5,150.0,1.0,0.006667,1.0,-14.024522
3,1.5,100.0,1.0,0.010000,1.0,-14.886890
4,1.5,80.0,1.0,0.012500,1.0,-15.562319
...,...,...,...,...,...,...
355,1.5,60.0,100000.0,1666.667000,100000.0,-7.749391
356,1.5,40.0,100000.0,2500.000000,100000.0,-8.099714
357,1.5,20.0,100000.0,5000.000000,100000.0,-8.647447
358,1.5,10.0,100000.0,10000.000000,100000.0,-9.023773
